In [32]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('Kangaroo.csv') # Load a CSV

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns


In [ ]:
df_columns = df.columns.tolist()
df_columns

In [ ]:
df.info() # Summary info of the data

In [ ]:
# Check for missing values
df.isnull().sum()

### Summary of infos collected
- Shape: 80,368 x 53
- Some columns are likely unnecessary: url, id?, unnamed,...
- Many columns misses a lot information:
100%: monthlyCost, hasBalcony, accessibleDisabledPeople

## Data cleaning
1. Remove duplicate : No duplicates in data set
2. Remove irrelevant columns: URL, Unnamed:0, 
3. Remove columns missing a lot info


In [ ]:
df.duplicated().sum() # Check for duplicate rows


### Removing Duplicates
Duplicates do not bring any new information and can bias statistics. We remove them first.

In [ ]:
# 2. Drop irrelevant columns
df_cleaned = df.drop(columns=["Unnamed: 0", "url"])
df_cleaned.shape
df_cleaned.head()


In [ ]:
#3. Trim Whitespace in Strings

str_cols = df_cleaned.select_dtypes(include='object').columns #Selects columns that are of type string or mixed object
for col in str_cols:
    df_cleaned[col] = df_cleaned[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

df_cleaned.describe(include='all') # Summary statistics of the data



### Remove rows with missing `price`
This column is essential for predictive modeling and analysis, so we discard rows without it.

In [ ]:
# Delete all rows with missing price
initial_shape = df_cleaned.shape
df_cleaned = df_cleaned.dropna(subset=['price'])
print(f"Deleted {initial_shape[0] - df_cleaned.shape[0]} rows without a price.")
print("New shape after dropping missing prices:", df_cleaned.shape)

### Drop columns with more than 90% missing values (except `hasSwimmingPool`)
Columns with more than 90% missing values are considered too sparse to be reliable. However, we retain `hasSwimmingPool` due to its interpretability and potential usefulness.

In [33]:
# Drop columns with >90% missing values, but keep 'hasSwimmingPool'
cols_to_drop = df_cleaned.columns[
    (df_cleaned.isnull().mean() > 0.9) & (df_cleaned.columns != 'hasSwimmingPool')
]
df_cleaned = df_cleaned.drop(columns=cols_to_drop)
print("Dropped columns:", list(cols_to_drop))

Dropped columns: ['monthlyCost', 'hasDressingRoom', 'diningRoomSurface', 'hasHeatPump', 'hasThermicPanels', 'hasBalcony', 'gardenOrientation', 'hasAirConditioning', 'hasArmoredDoor', 'hasFireplace', 'accessibleDisabledPeople']


### Imputing Remaining Missing Values
We now fill remaining missing values to avoid issues in later analysis:
- For **numerical columns**, we use the **median** to avoid the effect of outliers.
- For **categorical columns**, we use the **mode** (most frequent value).

In [ ]:
# Keep a copy of the dataset before imputing
df_before_impute = df_cleaned.copy()

for col in df_cleaned.select_dtypes(include='number').columns:
    if df_cleaned[col].isnull().sum() > 0:
        median_val = df_cleaned[col].median()
        df_cleaned[col] = df_cleaned[col].fillna(median_val)

for col in df_cleaned.select_dtypes(include='object').columns:
    if df_cleaned[col].isnull().sum() > 0:
        mode_val = df_cleaned[col].mode()[0]
        df_cleaned[col] = df_cleaned[col].fillna(mode_val)
        
df_cleaned = df_cleaned.infer_objects(copy=False)

In [ ]:
# Save the cleaned dataset
df_cleaned.to_csv('Kangaroo_cleaned.csv', index=False)
print("Cleaned dataset saved as 'Kangaroo_cleaned_presentation.csv'")

In [ ]:
duplicate_ids = df_cleaned['id'].duplicated().sum()
print(f"Found {duplicate_ids} duplicated IDs.")